# **Objetivo del notebook**

En esta etapa vamos a limpiar y preparar los datos: realizaremos una limpieza exhaustiva de los comentarios de texto, eliminando caracteres especiales, convirtiendo el texto a minúsculas y tokenizando las palabras.

Como estamos trabajando con textos y vamos entrenar un modelo de NLP, el preprocesamiento de los datos en este caso consistirá en distintos pasos de limpieza del texto para dejarlo listo para el modelo.



# **Contenido del Notebook**

1. Ingeniería de características
2. Eliminar columnas innecesarias
3. Regex: Limpieza y filtrado de texto
4. Stopwords: Eliminación de palabras vacías
5. Tokenización: Divide los comentarios en palabras o tokens.
6. Lematización: Normalización de las palabras
7. Vectorizar: ^_^ 
8. Codificación: Convertir Bool to int

In [ ]:
# Data analysis libraries
import numpy as np
import pandas as pd
import math
from scipy import stats
from tabulate import tabulate

# Data visualization libraries
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from wordcloud import WordCloud

In [ ]:
# read csv file
path = "/work/youtoxic_english_1000.csv"
df = pd.read_csv(path)

## **1. Ingeniería de características**

Inicialmente vamos a usar 'IsToxic' como variable objetivo.
En una siguiente vuelta de Scrum queremos probar a crear una nueva columna a partir de los 'True' de las ya existentes, y usarla como variable objetivo.

## **2. Eliminar columnas innecesarias**

## **3. Regex: Limpieza y filtrado de texto**

## **4. Stopwords: Eliminación de palabras vacías**

## **5. Tokenización: Divide los comentarios en palabras o tokens**

## **6. Lematización: Normalización de las palabras**

## **7. Vectorizar**

## **8. Codificación: Convertir Bool to int**

##

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=22be43cf-6e58-4d4e-8694-3183dc7644ab' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>